In [2]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.181, 0.074, 0.067, 0.101],
    [0.078, 0.055, 0.049, 0.093],
    [0.021, 0.069, 0.035, 0.09],
    [0.033, 0.078, 0.076, 0.08],
    [0.028, 0.081, 0.235, 0.071],
    [0.304, 0.052, 0.112, 0.062],
    [0.038, 0.143, 0.032, 0.131],
    [0.014, 0.095, 0.045, 0.134],
    [0.028, 0.144, 0.136, 0.095],
    [0.014, 0.076, 0.148, 0.081],
    [0.009, 0.132, 0.065, 0.063]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# Normalize the decision matrix
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# ELECTRE I method
def electre(matrix, criteria_types, weights, sites, criteria, c_thresh=0.6, d_thresh=0.4):
    n, m = matrix.shape
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Concordance matrix
    concordance = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            mask = weighted_matrix[i] >= weighted_matrix[j]
            concordance[i, j] = weights[mask].sum()

    # Discordance matrix
    discordance = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            diff = np.abs(weighted_matrix[i] - weighted_matrix[j])
            denom = diff.max() if diff.max() != 0 else 1
            disc = np.max([
                (weighted_matrix[j, k] - weighted_matrix[i, k]) / denom
                if weighted_matrix[i, k] < weighted_matrix[j, k] else 0
                for k in range(m)
            ])
            discordance[i, j] = disc

    # Outranking relation
    c_bar = c_thresh
    d_bar = d_thresh
    outranking = (concordance >= c_bar) & (discordance <= d_bar)

    # Kernel (non-dominated alternatives)
    kernel = []
    for i in range(n):
        if not outranking[:, i].any():
            kernel.append(sites[i])

    # Concordance dominance count
    concordance_dominance = outranking.sum(axis=1)

    # Net dominance = Dominance - Dominated
    net_dominance = outranking.sum(axis=1) - outranking.sum(axis=0)

    # Prepare DataFrame
    df = pd.DataFrame({
        'Site': sites,
        'Concordance Dominance': concordance_dominance,
        'Net Dominance': net_dominance,
        'Kernel Member': [site in kernel for site in sites]
    })
    df['Rank (Concordance)'] = df['Concordance Dominance'].rank(method='min', ascending=False).astype(int)
    df['Rank (Net Dominance)'] = df['Net Dominance'].rank(method='min', ascending=False).astype(int)
    df = df.set_index('Site')
    df = df.sort_values('Rank (Net Dominance)')

    return df, concordance, discordance, outranking

# Run ELECTRE I
result_df, concordance, discordance, outranking = electre(matrix, criteria_types, weights, sites, criteria)

# Print results
print("ELECTRE I Results:")
print(result_df)

print("\nConcordance Matrix:\n", pd.DataFrame(concordance, index=sites, columns=sites))
print("\nDiscordance Matrix:\n", pd.DataFrame(discordance, index=sites, columns=sites))
print("\nOutranking Relation Matrix:\n", pd.DataFrame(outranking, index=sites, columns=sites))


ELECTRE I Results:
        Concordance Dominance  Net Dominance  Kernel Member  \
Site                                                          
Site9                       7              7           True   
Site5                       6              6           True   
Site6                       2              2           True   
Site10                      3              2          False   
Site11                      2              1          False   
Site1                       2              0          False   
Site4                       2              0          False   
Site7                       0             -1          False   
Site8                       1             -2          False   
Site2                       0             -7          False   
Site3                       0             -8          False   

        Rank (Concordance)  Rank (Net Dominance)  
Site                                              
Site9                    1                     1  
Site5   